# Analyse de données textuelles avec python

Dans le cadre de ce notebook, nous allons analyser plusieurs jeux de données textuelles avec différentes approches :
    - création d'un wordcloud
    - extraction des mots
    - exploration de données textuelles
    - manipulation de dataframes pour les données textuelles
    - création d'un classifieur en utilisant des données textuelles

Pour toutes ces étapes, nous utiliserons de nombreuses bibliothèques dont principalement :
    - nltk
    - scikit-learn

## Créer un wordcloud à partir de données

On importe les bibliothèques nécessaires

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

On importe les données depuis une base sqlite<br>
Utilisez pandas pour récupérer la base database.sqlite

Créez une grande chaîne de caractères avec tous les textes<br>
Les textes se trouvent dans la colonne ExtractedBodyText

## Quelques bases en traitement des données textuelles

Pour commencer, on importe les bibliothèques dont on aura besoin dans cet exemple

In [2]:
# On utilise la bibliothèque nltk qui est la référence en terme de traitement de données textuelles
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.stem import PorterStemmer
import string
print("J'ai bien importé nltk et string")

J'ai bien importé nltk et string


In [12]:
# On utilise ensuite une bibliothèque de data visualisation matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
print("J'ai bien importé matplotlib")

J'ai bien importé matplotlib


In [3]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

Les étapes pour explorer des données textuelles :
    1. Charger les données
    2. Tokeniser
    3. Compter
    4. Eliminer les stopwords
    5. Normalisation du texte
    6. Utilisation des n-grams (suite de n mots)

On veut compter le nombre de mot. Comment faire ?

On vérifie la fréquence des mots

On voit qu'il y a un certain nombre de termes posant problème

Il s'agit de la ponctuation et des stopword

In [ ]:
from nltk.corpus import stopwords
import string

print(stopwords.words('french'))
print(len(stopwords.words('english')))
print(string.punctuation)

Essayez de supprimer la poctuation et les stopwords

On veut normaliser le texte un peu plus, on utilise pour cela le stemmer<br>
Essayez de voir ce que fait cette classe (on utilise .stel())

In [33]:
from nltk.stem import PorterStemmer

### Une autre approche : l'utilisation d'un dataframe

On peut aussi travailler sur le DataFrame complet<br>
On va commence par récupérer les Emails de la base SQL

Créez une variable WithPresident qui indique si le mot président est présent<br>
Affichez le nombre d'occurence

On crée une seconde variable donnant la longueur des emails (on peut utiliser .apply(len) pour calculer la longeur pour chaque observation du dataframe

### On peut représenter des histogrammes

Essayez de trouver des représentatios intéressantes

## Les n-grams

Une autre approche est de travailler sur des n-grams, c'est-à-dire des combiniasons de plusieurs mots

In [6]:
from nltk import ngrams

Créez des n-grams sans stopwords

Comment pourrait-on utiliser ces n-grams ?

## Utilisation du machine learning

Essayons de construire un premier classifieur pour les emails

On utilise scikit-learn et un modèle du type SVM

On va essayer de prédire si un SMS est un spam ou non

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import pandas as pd
import numpy as np

On récupère les données se trouvant dans SMSSpamCollection

In [5]:
sms=pd.read_table("./Data/SMSSpamCollection",names=["type","message"])

In [7]:
sms.head()

,type,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


On divise les données apprentissage / test en utilisant la fonction train_test_split de sklearn

In [9]:
from sklearn.model_selection import train_test_split

In [8]:
y=sms["type"]
x=sms["message"]

In [10]:
x_train,x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

On utilise une méthode classique de traitement de données textuelles<br>
TF-IDF term frequency-inverse document frequency<br>
Il s'agit d'une méthode de feature extraction qui permet d'extraire des fréquences à partir de textes<br>
TF fréquence du mot dans le texte<br>
IDF importance du mot dans le corpus complet<br>

On crée un objet puis on fait un fit_transform sur les données d'apprentissage et un transform sur les données test

In [12]:
vectorizer=TfidfVectorizer()

In [13]:
x_train_trans=vectorizer.fit_transform(x_train)

In [18]:
x_train_trans

<4457x7737 sparse matrix of type '<class 'numpy.float64'>'
	with 59099 stored elements in Compressed Sparse Row format>

On crée un modèle de classification, ici SVM

In [19]:
modele_svc=LinearSVC()
modele_svc.fit(x_train_trans,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [27]:
from sklearn.metrics import confusion_matrix

In [31]:
x_test_trans=vectorizer.transform(x_test)

y_test_predict=modele_svc.predict(x_test_trans)

confusion_matrix(y_test,y_test_predict)

array([[951,   2],
       [ 14, 148]], dtype=int64)

On va représenter les différents indices de qualité avec sklearn

In [44]:
def filtre_spam(message):
    return modele_svc.predict(vectorizer.transform(np.array([message])))[0]

In [45]:
filtre_spam("free")

'spam'

Quelle est l'efficacité de ce classifieur ?

Essayez avec un random forrest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

...